In [1]:
import requests

In [16]:
URL="https://app-aiml24.cloud.sdu.dk/predict"
TEST_EMAIL = "test@cbs.dk"

***
***

In [19]:
class UCloudRequester:
    """ This class is a simple container for the UCloud API 
    request. It is used to send a query to the UCloud API 
    and return the response with or without metadata."""

    def __init__(self, url : str, email : str) -> None:
        self.url = url # URL of the UCloud API
        self.email = email # Email of the user

    def _predict(self, prompt : str, **params) -> dict:
        """This method handles the call to the UCloud API and
        returns the response as a dictionary"""

        try:
            query = {
                'email': self.email,
                'prompt': prompt,
                'params': {**params}
            }
            response = requests.post(
                url=self.url,
                headers={'Content-Type': 'application/json'},
                json=query
            )
            return response.json()
        
        except requests.exceptions.JSONDecodeError:
            return f"JSONDecodeError: Failed to decode response as JSON: {response.text}"

        except requests.exceptions.HTTPError as e:
            return "HTTPError: " + str(e)
        
    def predict(self, prompt : str, return_meta=False, **params) -> dict | str:
        """This method is the main method for sending a request.
        if return_meta is set to True, the method will return the
        full response as a dictionary. Otherwise, it will return
        the text of the first choice in the response."""

        result : dict = self._predict(prompt, **params)
        if return_meta:
            return result
        return result["choices"][0]["text"]
    

requester = UCloudRequester(
    url=URL,
    email=TEST_EMAIL
)

***
***

In [20]:
# Example model parameters
params = dict(
    max_tokens=1000,
    temperature=0.5,
    stop=[],
)

In [21]:
PROMPT = """Malaysia Airlines Flight 370 (MH370/MAS370)[a] was an international passenger flight operated by Malaysia Airlines that disappeared from radar on 8 March 2014 while flying from Kuala Lumpur International Airport in Malaysia to its planned destination, Beijing Capital International Airport in China.[1] The reason for its disappearance has not been determined.
The crew of the Boeing 777-200ER, registered as 9M-MRO, last communicated with air traffic control (ATC) around 38 minutes after takeoff when the flight was over the South China Sea. The aircraft was lost from ATC's secondary surveillance radar screens minutes later, but was tracked by the Malaysian military's primary radar system for another hour, deviating westward from its planned flight path, crossing the Malay Peninsula and Andaman Sea. It left radar range 200 nautical miles (370 km; 230 mi) northwest of Penang Island in northwestern Peninsular Malaysia.
With all 227 passengers and 12 crew aboard presumed dead, the disappearance of Flight 370 was the deadliest incident involving a Boeing 777 and the deadliest in Malaysia Airlines' history until it was surpassed in both regards by Malaysia Airlines Flight 17, which was shot down while flying over Ukraine four months later on 17 July 2014. The combined loss caused significant financial problems for Malaysia Airlines, which was renationalised by the Malaysian government in August 2014.
The search for the missing airplane became the most expensive search in the history of aviation. It focused initially on the South China Sea and Andaman Sea, before a novel analysis of the aircraft's automated communications with an Inmarsat satellite indicated that the plane had traveled far southward over the southern Indian Ocean. The lack of official information in the days immediately after the disappearance prompted fierce criticism from the Chinese public, particularly from relatives of the passengers, as most people on board Flight 370 were of Chinese origin. Several pieces of debris washed ashore in the western Indian Ocean during 2015 and 2016; many of these were confirmed to have originated from Flight 370. What is the name the company that owned Flight 370? 
"""

In [22]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [23]:
prediction

{'id': 'cmpl-42f1e68f-dfb1-46e5-9e85-56ac49b9636c',
 'object': 'text_completion',
 'created': 1710269509,
 'model': 'models/mixtral-8x7b-instruct-v0.1.Q6_K.gguf',
 'choices': [{'text': '\nMalaysia Airlines',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 516, 'completion_tokens': 5, 'total_tokens': 521},
 'timestamp': '2024-03-12T18:51:51.084399',
 'email': 'test@cbs.dk'}

In [24]:
PROMPT = """Write a short poem about the meaning of life.
Poem:

"""

In [25]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [26]:
print(prediction["choices"][0]["text"])

In the vast expanse of cosmic time,
A fleeting moment is our prime.
To find our purpose, seek and strive,
Is the essence of this mortal life.

Through love, we connect and unite,
Cherishing moments in soft candlelight.
With kindness, we heal wounds so deep,
And together, in dreams, we leap.

Through struggle, we learn and grow,
Embracing change as we flow.
With courage, we face each new day,
Finding strength in our own gentle way.

So let us cherish every breath,
And live with joy till our last breath.
For in this dance, we find our song,
And in each other, we belong.


In [23]:
%%timeit -n 10 -r 10

requester.predict(prompt=PROMPT, **params)

3.79 s ± 216 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [68]:
%%timeit -n 10 -r 10

requester.predict(f"The meaning of life is", **params)

741 ms ± 134 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
